# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [4]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = None, names = columns))


# Get X and Y

Create the features data frame and target data.

In [5]:
%load_ext dotenv
%dotenv 
import os
import sys
sys.path.append(os.getenv('SRC_DIR')) 
import pandas as pd
import numpy as np
from logger import get_logger 
_logs = get_logger(__name__)

In [7]:
from sklearn.model_selection import train_test_split

X = fires_dt.drop(columns='area')
Y = fires_dt['area']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


In [25]:
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV 


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [26]:
# preproc1

num_cols = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
cat_cols = ['month', 'day']

preproc1 = ColumnTransformer([  
    ('num', StandardScaler(), num_cols),
 ('cat', OneHotEncoder (handle_unknown= 'ignore', drop= 'first'), cat_cols)])
preproc1


ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore'),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [97]:
# preproc2
from sklearn.preprocessing import PowerTransformer

preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', StandardScaler()),
            ('boxcox', PowerTransformer(method='yeo-johnson'))
        ]), num_cols),
        ('cat', OneHotEncoder(drop='first'), cat_cols)
    ])

preproc2

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler()),
                                                 ('boxcox',
                                                  PowerTransformer())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat', OneHotEncoder(drop='first'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [98]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

scoring = ['neg_log_loss', 'roc_auc', 'f1', 'accuracy', 'precision', 'recall']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [101]:
# Pipeline A = preproc1 + baseline

pipe_a = Pipeline ([
    ('preprocess', preproc1), 
    ('regressor', LogisticRegression() )
])

pipe_a.get_params()


{'memory': None,
 'steps': [('preprocess',
   ColumnTransformer(transformers=[('num', StandardScaler(),
                                    ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                     'isi', 'temp', 'rh', 'wind', 'rain']),
                                   ('cat',
                                    OneHotEncoder(drop='first',
                                                  handle_unknown='ignore'),
                                    ['month', 'day'])])),
  ('regressor', LogisticRegression())],
 'verbose': False,
 'preprocess': ColumnTransformer(transformers=[('num', StandardScaler(),
                                  ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                   'isi', 'temp', 'rh', 'wind', 'rain']),
                                 ('cat',
                                  OneHotEncoder(drop='first',
                                                handle_unknown='ignore'),
                                  ['mo

In [102]:
# Pipeline B = preproc2 + baseline

pipe_b = Pipeline ([
    ('preprocess', preproc2), 
    ('regressor', LogisticRegression() )
])


In [112]:
# Pipeline C = preproc1 + advanced model
pipe_c = Pipeline ([
    ('preprocess', preproc1), 
    ('classifier',RandomForestClassifier(random_state= 42)  )
])

pipe_a.get_params()

{'memory': None,
 'steps': [('preprocess',
   ColumnTransformer(transformers=[('num', StandardScaler(),
                                    ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                     'isi', 'temp', 'rh', 'wind', 'rain']),
                                   ('cat',
                                    OneHotEncoder(drop='first',
                                                  handle_unknown='ignore'),
                                    ['month', 'day'])])),
  ('regressor', LogisticRegression())],
 'verbose': False,
 'preprocess': ColumnTransformer(transformers=[('num', StandardScaler(),
                                  ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                   'isi', 'temp', 'rh', 'wind', 'rain']),
                                 ('cat',
                                  OneHotEncoder(drop='first',
                                                handle_unknown='ignore'),
                                  ['mo

In [114]:
# Pipeline D = preproc2 + advanced model
pipe_d = Pipeline ([
    ('preprocess', preproc2), 
    ('classifier',RandomForestClassifier(random_state= 42)  )
])

pipe_d.get_params()

{'memory': None,
 'steps': [('preprocess',
   ColumnTransformer(transformers=[('num',
                                    Pipeline(steps=[('scaler', StandardScaler()),
                                                    ('boxcox',
                                                     PowerTransformer())]),
                                    ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                     'isi', 'temp', 'rh', 'wind', 'rain']),
                                   ('cat', OneHotEncoder(drop='first'),
                                    ['month', 'day'])])),
  ('classifier', RandomForestClassifier(random_state=42))],
 'verbose': False,
 'preprocess': ColumnTransformer(transformers=[('num',
                                  Pipeline(steps=[('scaler', StandardScaler()),
                                                  ('boxcox',
                                                   PowerTransformer())]),
                                  ['coord_x', 'coord_y', '

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [109]:
#Pipeline_A
param_grid = { 
    'regressor__C': [0.01, 0.5, 1.0],
    'regressor__penalty': ['l1','l2'],
    'regressor__solver': ['liblinear'], 
    'regressor__fit_intercept' : [True]
    }

grid_cv = GridSearchCV(
    estimator=pipe_a,
    param_grid=param_grid, 
    scoring = scoring, 
    cv = 5,
    refit = "neg_log_loss") 
grid_cv.fit(X_train, Y_train)

res = grid_cv.cv_results_  
res = pd.DataFrame(res)
res.columns

res[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 
       'param_regressor__C', 'param_regressor__penalty', 'param_regressor__solver','param_regressor__fit_intercept', 'params',
       'mean_test_neg_log_loss',
       'std_test_neg_log_loss', 'rank_test_neg_log_loss']].sort_values('rank_test_neg_log_loss')



c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__C,param_regressor__penalty,param_regressor__solver,param_regressor__fit_intercept,params,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss
0,0.123282,0.028627,0.150737,0.045060,0.01,l1,liblinear,True,"{'regressor__C': 0.01, 'regressor__fit_interce...",NaN,NaN,1
1,0.293400,0.100761,0.127878,0.033573,0.01,l2,liblinear,True,"{'regressor__C': 0.01, 'regressor__fit_interce...",NaN,NaN,1
2,0.160503,0.050100,0.125707,0.045376,0.50,l1,liblinear,True,"{'regressor__C': 0.5, 'regressor__fit_intercep...",NaN,NaN,1
3,0.443834,0.065404,0.149362,0.048755,0.50,l2,liblinear,True,"{'regressor__C': 0.5, 'regressor__fit_intercep...",NaN,NaN,1
4,0.274252,0.134248,0.134798,0.032961,1.00,l1,liblinear,True,"{'regressor__C': 1.0, 'regressor__fit_intercep...",NaN,NaN,1
5,0.393700,0.134876,0.196241,0.121419,1.00,l2,liblinear,True,"{'regressor__C': 1.0, 'regressor__fit_intercep...",NaN,NaN,1


In [108]:
#Pipeline_B
param_grid2 = {
    'regressor__C': [0.01, 0.5, 1.0],
    'regressor__penalty': ['l1', 'l2'],
    'regressor__solver': ['liblinear'],
    'regressor__fit_intercept': [True]
}

grid_cv2 = GridSearchCV(
    estimator=pipe_b,
    param_grid=param_grid2,  
    scoring=scoring,
    cv=5,
    refit="neg_log_loss"
)

grid_cv2.fit(X_train, Y_train)

res2 = grid_cv2.cv_results_  
res2 = pd.DataFrame(res2)
res2.columns

res2[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 
       'param_regressor__C', 'param_regressor__penalty', 'param_regressor__solver','param_regressor__fit_intercept', 'params',
       'mean_test_neg_log_loss',
       'std_test_neg_log_loss', 'rank_test_neg_log_loss']].sort_values('rank_test_neg_log_loss')


c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__C,param_regressor__penalty,param_regressor__solver,param_regressor__fit_intercept,params,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss
0,0.310394,0.088583,0.182000,0.027863,0.01,l1,liblinear,True,"{'regressor__C': 0.01, 'regressor__fit_interce...",NaN,NaN,1
1,0.768252,0.342448,0.286426,0.167463,0.01,l2,liblinear,True,"{'regressor__C': 0.01, 'regressor__fit_interce...",NaN,NaN,1
2,0.322529,0.159174,0.217397,0.154495,0.50,l1,liblinear,True,"{'regressor__C': 0.5, 'regressor__fit_intercep...",NaN,NaN,1
3,0.280107,0.118605,0.064115,0.019690,0.50,l2,liblinear,True,"{'regressor__C': 0.5, 'regressor__fit_intercep...",NaN,NaN,1
4,0.330109,0.069467,0.153047,0.065034,1.00,l1,liblinear,True,"{'regressor__C': 1.0, 'regressor__fit_intercep...",NaN,NaN,1
5,0.358559,0.052500,0.096466,0.017143,1.00,l2,liblinear,True,"{'regressor__C': 1.0, 'regressor__fit_intercep...",NaN,NaN,1


In [127]:
#Pipeline_C

param_grid3 = { 
    'classifier__n_estimators': [50, 100, 200],  
    'classifier__max_depth': [None],  
    'classifier__min_samples_split': [2], 
    'classifier__min_samples_leaf': [1]
}

grid_cv3 = GridSearchCV(
    estimator=pipe_c,
    param_grid=param_grid3, 
    scoring = scoring, 
    cv = 5,
    refit = "neg_log_loss") 
grid_cv3.fit(X_train, Y_train)

res3 = grid_cv3.cv_results_  
res3 = pd.DataFrame(res)
res3.columns

res3[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 
       'mean_test_neg_log_loss',
       'std_test_neg_log_loss', 'rank_test_neg_log_loss']].sort_values('rank_test_neg_log_loss')

c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss
0,0.123282,0.028627,0.150737,0.045060,NaN,NaN,1
1,0.293400,0.100761,0.127878,0.033573,NaN,NaN,1
2,0.160503,0.050100,0.125707,0.045376,NaN,NaN,1
3,0.443834,0.065404,0.149362,0.048755,NaN,NaN,1
4,0.274252,0.134248,0.134798,0.032961,NaN,NaN,1
5,0.393700,0.134876,0.196241,0.121419,NaN,NaN,1


In [128]:
# Pipeline D
param_grid4 = { 
    'classifier__n_estimators': [50, 100, 200],  
    'classifier__max_depth': [None],  
    'classifier__min_samples_split': [2], 
    'classifier__min_samples_leaf': [1]
}

grid_cv4 = GridSearchCV(
    estimator=pipe_c,
    param_grid=param_grid4, 
    scoring = scoring, 
    cv = 5,
    refit = "neg_log_loss") 
grid_cv4.fit(X_train, Y_train)

res4 = grid_cv4.cv_results_  
res4 = pd.DataFrame(res)
res4.columns

res4[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 
       'mean_test_neg_log_loss',
       'std_test_neg_log_loss', 'rank_test_neg_log_loss']].sort_values('rank_test_neg_log_loss')

c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "c:\Users\sds70\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss
0,0.123282,0.028627,0.150737,0.045060,NaN,NaN,1
1,0.293400,0.100761,0.127878,0.033573,NaN,NaN,1
2,0.160503,0.050100,0.125707,0.045376,NaN,NaN,1
3,0.443834,0.065404,0.149362,0.048755,NaN,NaN,1
4,0.274252,0.134248,0.134798,0.032961,NaN,NaN,1
5,0.393700,0.134876,0.196241,0.121419,NaN,NaN,1


# Evaluate

+ Which model has the best performance?

In [ ]:
# They all have similar performances in terms of mean_fit_time and mean_score_time given the hyperparams tuning above. 
# I am going with pipeline A becaues it's the simplest .

# Export

+ Save the best performing model to a pickle file.

In [190]:
import pickle
grid_cv.best_params_
pipe_a.set_params(**grid_cv.best_params_) 
best_pipe = grid_cv.best_estimator_



In [192]:
file_path = r'C:\Users\sds70\Desktop\DSI\scaling_to_production\02_activities\assignments\best_pipe.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(best_pipe, file)


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [212]:
import shap

file_path = r'C:\Users\sds70\Desktop\DSI\scaling_to_production\02_activities\assignments\best_pipe.pkl'

with open(file_path, 'rb') as file:
    pipe = pickle.load(file)

data_transform = pipe.named_steps['preprocess'].transform(X_train) 

explainer = shap.explainers.Linear( 
    pipe.named_steps['regressor'],  
    data_transform,
    feature_names = pipe.named_steps['preprocess'].get_feature_names_out())

shap_values = explainer(data_transform)

pipe.named_steps
pipe.named_steps ['preprocess'].transform (X_train) 


array([[ 1.4463699 ,  1.39745997,  0.12623934, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.02034394, -0.22916546,  0.16778356, ...,  0.        ,
         0.        ,  0.        ],
       [-1.10978585,  0.58414726,  0.16778356, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.25773394, -0.22916546,  0.2716441 , ...,  0.        ,
         1.        ,  0.        ],
       [-1.53581181, -0.22916546, -0.03993754, ...,  0.        ,
         0.        ,  0.        ],
       [-0.68375989, -0.22916546, -0.4138355 , ...,  0.        ,
         1.        ,  0.        ]])

In [213]:
shap.plots.waterfall(shap_values[1])

ValueError: The waterfall plot can currently only plot a single explanation, but a matrix of explanations (shape (27, 181)) was passed! Perhaps try `shap.plots.waterfall(shap_values[0])` or for multi-output models, try `shap.plots.waterfall(shap_values[0, 0])`.

In [214]:
shap.plots.beeswarm(shap_values)

ValueError: The beeswarm plot does not support plotting explanations with instances that have more than one dimension!

In [ ]:
# my code for waterfall plot and beeswarm plot did not work - I am unable to debug. 

#

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.